### PPOCR 형식의 STD 데이터 셋을 PPOCR 형식 STR 데이터 셋으로 변환
1. 이미지에서 텍스트만 잘라서 저장
2. 레이블 파일 생성

In [9]:
from dataset import *
from tools import polygon_utility

def make_and_save_recognition_dataset(dataset, save_dir, dir_size=1000, label_file_name = "label.txt"):
    assert not save_dir.exists(), f"please remove {save_dir}"
    label_path = save_dir/label_file_name
    image_idx = 0

    for i in range(len(dataset)):  
        try:
            image = dataset.get_x(i)["image"]
            labels = dataset.get_y(i)
    
            for label in labels:
                
                polygon =  label["polygon"]
                text = label["text"]
                
                cropped_image = polygon_utility.crop_by_polygon(image, polygon)
                image_path = save_dir/f"{(image_idx//dir_size + 1)}"/f"{image_idx+1}.png"
                image_path.parent.mkdir(parents=True, exist_ok=True)
                cropped_image.save(image_path)
                with open(label_path, "a") as f:
                    image_name = str(image_path.relative_to(save_dir)).replace('\\', '/')
                    f.write(f"{image_name}\t{text}\n")

                image_idx += 1
        except:
            continue

In [10]:
order = [["sangmu_std_MH", "sangmu_str_MH"],
        ["sangmu_std_DI", "sangmu_str_DI"],
        ["sangmu_std_DH", "sangmu_str_DH"]]

order = [["sangmu_std_MH", "sangmu_str_MH_v2"],
        ["sangmu_std_DI", "sangmu_str_DI_v2"],
        ["sangmu_std_DH", "sangmu_str_DH_v2"]]

for _std, _str in order:
    dataset = PPOCR_STD_Dataset_Loader(f"/home/datasets/{_std}", label_file_name="label.txt")

    save_dir = Path(f"/home/datasets/{_str}")
    make_and_save_recognition_dataset(dataset, save_dir, dir_size=1000, label_file_name="label.txt")


IBK WM 센터 광주
IBK 기업은행
IBK 투자증권
뷰티스 맑은 피부과의원
피부과
전문의
LGU+
SPAO
광주 스마일 안과 의원
TIMES
TOWER
Iet
롯데손해보험
신영증권
현대리바트
HYUND
LIVAR TKITCHEN
광주전시장Tel385-6093
상무스타치과병원
교보문고
cafezal
신영증권
HUG
주택도시보증공사
모던성형외과의원
현대리바트
LLVARTKITCHEN
HYOND
SK
에너지
Kt
한국수출입은행
KYOBO
교보문고
한국투자저축은행
KMI
한국의학연구소
상무스타치과병원
탑라인외과
의원
진료과목
성형외과
광주스마일안과의원
IBKWM센터광주
IBK기업은행
IBK투자증권
교보문고
Cafezal
UP TOWN HOLIC
페퍼저축은행
Pepper
UNICO
JOIL
D&D
JOIL
D&D
UNICO
BALLISTIC
GOLF
B
생생약국
노블레스수현 결혼정보
광추지사2F716.5080
전자담배
7번가피자
1688-7887
상무점
374-7776
PIZZA
STRRET
th
7
PIZ
STREET
th
SALAD
FRUIT
JUICE
상무점
뱃살도둑
BODYFRIEND
Daishin Securities
대신증권
상무WM센터 2F
바디프랜드
BODYFRIEND
제이엘치과의원
압구정제이엘의원
진료과목
피부과
모던성형외과의원
True
friend
한국투자저축은행
HIP'S
Happiness In Pharmacy
상무중앙약국
COWAY BEREX
한국수출입은행
탑라인외과
의원
진료과목
성형외과
비슈정신건강의학과의원
한국의학연구소
광주의원
SPAO
교보문고
HOTTRACKS
V라인 S라인 프리미엄자인의원
플라잉필라테스요가
574-0724
요가포레스트
T.574-0724
다비치안경 콘택트
Davich Optical Chain
정액정찰제
1|3|5|7|9만원
다비치안경 콘택트
Dabich Optical Chain
정액정찰제
1|3|5|7|9만원
V라인 S라인 프리미엄 자인의원
플라잉 필라테스 요가
574-0724
V라인 S라인 프리미엄 자인의원
플라잉 필